<a href="https://colab.research.google.com/github/Yiifeiii/IT1244/blob/main/Early_stage_vs_Healthy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is done on healthy vs early stage cancer dataset.

Contents:
1. Data - import data, filter healthy and early stage cancer classes
2. Feature selection - PCA
3. Logistic Regression (PCA)
4. Random Forest - Smote and API
5. SVM - GridSearchCV

## 1. Data

1a. Import Data

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/Train_Set.csv')

# TEST DATA
dftest = pd.read_csv('/content/Test_Set.csv')
# Display the DataFrame
print(df)

      length_51  length_52  length_53  length_54  length_55  length_56  \
0      0.000152   0.000081   0.000087   0.000092   0.000099   0.000125   
1      0.006256   0.006413   0.006512   0.006469   0.006810   0.007070   
2      0.003783   0.003886   0.004063   0.004237   0.004481   0.004832   
3      0.004635   0.004471   0.004383   0.004829   0.004920   0.005056   
4      0.011315   0.010045   0.009795   0.009906   0.010630   0.011467   
...         ...        ...        ...        ...        ...        ...   
2188   0.007355   0.007079   0.007045   0.007455   0.007724   0.008320   
2189   0.007630   0.007204   0.007112   0.007522   0.007885   0.008424   
2190   0.007663   0.007429   0.007198   0.007816   0.008329   0.008497   
2191   0.008022   0.007604   0.007436   0.008037   0.008367   0.008986   
2192   0.008126   0.007708   0.007657   0.008192   0.008471   0.008952   

      length_57  length_58  length_59  length_60  ...  length_392  length_393  \
0      0.000122   0.000115   0

1b. Create a filtered dataframe with only healthy and early stage cancer

In [ ]:
# Define the classes you want to filter out
classes_to_filter = ['mid stage cancer', 'late stage cancer', 'screening stage cancer']

# Filter out rows where the 'class_label' column contains classes to filter out
filtered_df = df[~df['class_label'].isin(classes_to_filter)]
filtered_dftest = dftest[~dftest['class_label'].isin(classes_to_filter)]

# Display the filtered DataFrame
print(filtered_df)
print(filtered_dftest)
print(filtered_df['class_label'].unique())
print(filtered_dftest['class_label'].unique())

      length_51  length_52  length_53  length_54  length_55  length_56  \
0      0.000152   0.000081   0.000087   0.000092   0.000099   0.000125   
1      0.006256   0.006413   0.006512   0.006469   0.006810   0.007070   
2      0.003783   0.003886   0.004063   0.004237   0.004481   0.004832   
3      0.004635   0.004471   0.004383   0.004829   0.004920   0.005056   
4      0.011315   0.010045   0.009795   0.009906   0.010630   0.011467   
...         ...        ...        ...        ...        ...        ...   
2188   0.007355   0.007079   0.007045   0.007455   0.007724   0.008320   
2189   0.007630   0.007204   0.007112   0.007522   0.007885   0.008424   
2190   0.007663   0.007429   0.007198   0.007816   0.008329   0.008497   
2191   0.008022   0.007604   0.007436   0.008037   0.008367   0.008986   
2192   0.008126   0.007708   0.007657   0.008192   0.008471   0.008952   

      length_57  length_58  length_59  length_60  ...  length_392  length_393  \
0      0.000122   0.000115   0

1c. Remove NA rows from filtered data

In [ ]:
rows_before = filtered_df.shape[0]

# Remove rows with NA = df.dropna()
filtered_df = filtered_df.dropna()

# Count the number of rows after dropping NA values
rows_after = filtered_df.shape[0]

# Calculate the difference in row counts
na_rows_dropped = rows_before - rows_after

# Display the count of NA rows dropped
print("Number of NA rows dropped for filtered dataframe:", na_rows_dropped)



Number of NA rows dropped for filtered dataframe: 0


1d. Labelling data

In [ ]:
# Drop missing values for training data
filtered_df = filtered_df.dropna()

# Drop missing values for test data
filtered_dftest = filtered_dftest.dropna()

filtered_df.dropna(inplace=True)
filtered_dftest.dropna(inplace=True)

# Separate features and class label for training data
X_filtered = filtered_df.drop(columns=['class_label'])
y_filtered = filtered_df['class_label']

# Separate features and class label for test data
X_filtered_test = filtered_dftest.drop(columns=['class_label'])
y_filtered_test = filtered_dftest['class_label']


1e. Normalizing TRAIN and TEST data for FILTERED dataframe

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Normalize the features for training data
filtered_X_normalized = scaler.fit_transform(X_filtered)

# Normalize the features for test data
filtered_X_normalized_test = scaler.transform(X_filtered_test)

# Create DataFrames with normalized features for training and test data
filtered_df_normalized = pd.DataFrame(filtered_X_normalized, columns=X_filtered.columns)
filtered_df_normalized['class_label'] = y_filtered.values

filtered_dftest_normalized = pd.DataFrame(filtered_X_normalized_test, columns=X_filtered_test.columns)
filtered_dftest_normalized['class_label'] = y_filtered_test.values

# Display the filtered and normalized DataFrames for training and test data
print(filtered_df_normalized)
print(filtered_dftest_normalized)

     length_51  length_52  length_53  length_54  length_55  length_56  \
0     0.007378   0.003382   0.004230   0.004221   0.003644   0.005164   
1     0.383494   0.416609   0.422897   0.418944   0.420269   0.416458   
2     0.231073   0.251698   0.263312   0.273806   0.275724   0.283950   
3     0.283586   0.289851   0.284208   0.312290   0.302936   0.297171   
4     0.695197   0.653602   0.636861   0.642499   0.657474   0.676854   
..         ...        ...        ...        ...        ...        ...   
836   0.451174   0.460034   0.457684   0.483043   0.477062   0.490486   
837   0.468173   0.468209   0.462008   0.487399   0.487006   0.496670   
838   0.470203   0.482887   0.467593   0.506560   0.514602   0.500967   
839   0.492290   0.494324   0.483150   0.520920   0.516954   0.529920   
840   0.498693   0.501108   0.497517   0.531025   0.523405   0.527917   

     length_57  length_58  length_59  length_60  ...  length_392  length_393  \
0     0.004483   0.003160   0.004138   0.00

## 2. Feature Selection (PCA)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt

# Standardize the features
scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X_filtered)
X_scaled_filtered = scaler.fit_transform(filtered_df_normalized.drop(columns=['class_label']))
# Initialize PCA
pca = PCA(n_components=10) #Choose no. of PCs

# Perform PCA
X_pca = pca.fit_transform(X_scaled_filtered)


In [ ]:
print(X_pca)

## 3. Baseline Model: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import classification_report, roc_auc_score
# Standardize the features for filtered data
max_abs_scaler_filtered = MaxAbsScaler()
X_train_scaled_filtered = max_abs_scaler_filtered.fit_transform(X_filtered)
X_test_scaled_filtered = max_abs_scaler_filtered.transform(X_filtered_test)

# Initialize PCA for filtered data
pca_filtered = PCA(n_components=10)  # You can choose the number of components as needed

# Perform PCA for filtered data
X_train_pca_filtered = pca_filtered.fit_transform(X_train_scaled_filtered)
X_test_pca_filtered = pca_filtered.transform(X_test_scaled_filtered)

# Initialize Logistic Regression model for filtered data
log_reg_model_filtered = LogisticRegression()

# Train the model on the training data for filtered data
log_reg_model_filtered.fit(X_train_pca_filtered, y_filtered)

# Predict on the testing data for filtered data
y_pred_filtered = log_reg_model_filtered.predict(X_test_pca_filtered)
y_pred_proba_filtered = log_reg_model_filtered.predict_proba(X_test_pca_filtered)[:, 1]
# Calculate accuracy for filtered data
accuracy_filtered = accuracy_score(y_filtered_test, y_pred_filtered)
print("Accuracy for filtered data:", accuracy_filtered)
log_evaluation_metrics = classification_report(y_filtered_test, y_pred_filtered, target_names=['Healthy', 'Early Stage Cancer'], output_dict=True)
log_auc = roc_auc_score(y_filtered_test, y_pred_proba_filtered)

print(log_evaluation_metrics)
print('AUROC: '+str(log_auc))

Accuracy for filtered data: 0.8850855745721271
{'Healthy': {'precision': 0.9002493765586035, 'recall': 0.9809782608695652, 'f1-score': 0.9388816644993497, 'support': 368}, 'Early Stage Cancer': {'precision': 0.125, 'recall': 0.024390243902439025, 'f1-score': 0.04081632653061225, 'support': 41}, 'accuracy': 0.8850855745721271, 'macro avg': {'precision': 0.5126246882793017, 'recall': 0.5026842523860021, 'f1-score': 0.489848995514981, 'support': 409}, 'weighted avg': {'precision': 0.8225348913779121, 'recall': 0.8850855745721271, 'f1-score': 0.848855554825222, 'support': 409}}
AUROC: 0.8083907741251325


Accuracy = 0.6969

Precision: 0.72 for Healthy, 0.69 for Early stage

Recall: 0.32 for Healthy, 0.93 for Early Stage

F1-score: 0.44 for Healthy, 0.79 for Early Stage

AUROC: 0.84

## 4. Random Forest (no PCA!)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

4a. Applying random forest on standardized data

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_scaled_filtered, y_filtered)
y_pred = rf_classifier.predict(X_test_scaled_filtered)
y_pred_proba = rf_classifier.predict_proba(X_test_scaled_filtered)[:, 1]  # Probability estimates for the positive class

# Evaluating the initial Random Forest model without addressing imbalance explicitly
rf_evaluation_metrics = classification_report(y_filtered_test, y_pred, target_names=['Healthy', 'Early Stage Cancer'], output_dict=True)
rf_auc = roc_auc_score(y_filtered_test, y_pred_proba)

print(rf_evaluation_metrics)
print('AUROC: '+str(rf_auc))


Healthy: {Precision: 1.0, Recall: 0.52, F1-score: 0.68, Support: 41}

Early Stage: {Precision: 0.77, Recall: 1.0, F1-score: 0.87, Support: 41}

Accuracy: 0.82
**bold text**
AUROC: 0.95

4b. Applying SMOTE to improve representation for minority class

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.pipeline import Pipeline as ImblearnPipeline

In [ ]:
# Class count before SMOTE
class_counts_before_smote = Counter(y_filtered)

print("Number of classes before applying SMOTE:")
for class_label, count in class_counts_before_smote.items():
    print(f"Class {class_label}: {count} samples")

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled_filtered, y_filtered)

# Count the number of classes after SMOTE
class_counts_after_smote = Counter(y_train_smote)

print("Number of classes after applying SMOTE:")
for class_label, count in class_counts_after_smote.items():
    print(f"Class {class_label}: {count} samples")

Number of classes before applying SMOTE:
Class healthy: 60 samples
Class early stage cancer: 781 samples
Number of classes after applying SMOTE:
Class healthy: 781 samples
Class early stage cancer: 781 samples


Number of classes before applying SMOTE:
Class healthy: 60 samples
Class early stage cancer: 21 samples
Number of classes after applying SMOTE:
Class healthy: 60 samples
Class early stage cancer: 60 samples

In [ ]:
# Training the RandomForestClassifier with SMOTE to address class imbalance
smote_pipeline = ImblearnPipeline([
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])
#smote_pipeline.fit(X_train_scaled_filtered, y_filtered)
smote_pipeline.fit(X_train_scaled_filtered, y_filtered)
y_pred_smote = smote_pipeline.predict(X_test_scaled_filtered)
y_pred_proba_smote = smote_pipeline.predict_proba(X_test_scaled_filtered)[:, 1]

# Evaluating the SMOTE-improved Random Forest model
#precision = precision_score(y_filtered_test, y_pred_smote,pos_label='early stage cancer')
#recall = recall_score(y_filtered_test, y_pred_smote,pos_label='early stage cancer')
#f1 = f1_score(y_filtered_test, y_pred_smote,pos_label='early stage cancer')
smote_evaluation_metrics = classification_report(y_filtered_test, y_pred_smote, target_names=['Healthy', 'Early Stage Cancer'], output_dict=True)
smote_auc = roc_auc_score(y_filtered_test, y_pred_proba_smote)
print(smote_auc)
#print("Precision:", precision)
#print("Recall:", recall)
#print("F1-score:", f1)

(smote_evaluation_metrics, smote_auc)

AUROC: 0.96

Accuracy: 0.85

Healthy: {Precision: 1.0, Recall: 0.6, F1-score: 0.75, Support: 25}

Early Stage: {Precision: 0.80, Recall: 1.0, F1-score: 0.89, Support: 41}

4c. Applying API function to random forest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
# Define the parameter grid? (a dictionary where keys are the hyperparameter names and values are lists of the parameter values to try)
param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]}

rf = RandomForestClassifier(random_state=42, class_weight='balanced')

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_scaled_filtered, y_filtered)

# Access the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Use the best model
best_rf = RandomForestClassifier(**grid_search.best_params_)
best_rf.fit(X_scaled_filtered, y_filtered)

# Predict on the test data
y_pred_api = best_rf.predict(X_test_scaled_filtered)

# Calculate accuracy
accuracy = accuracy_score(y_filtered_test, y_pred_api)
print("Random Forest Accuracy for decision tree with API function:", accuracy)

Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}

Best Score: 0.9625

Random Forest Accuracy for decision tree with API function: 0.6212121212121212

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
y_pred_proba_api = best_rf.predict_proba(X_test_scaled_filtered)[:, 1]
auroc = roc_auc_score(y_filtered_test, y_pred_proba_api)
print("AUROC:", auroc)
#for early stage cancer
precision = precision_score(y_filtered_test, y_pred_api,pos_label='early stage cancer')
recall = recall_score(y_filtered_test, y_pred_api,pos_label='early stage cancer')
f1 = f1_score(y_filtered_test, y_pred_api,pos_label='early stage cancer')
api_evaluation_metrics = classification_report(y_filtered_test, y_pred_api, target_names=['Healthy', 'Early Stage Cancer'], output_dict=True)
print(api_evaluation_metrics)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


AUROC: 0.6673170731707316

'Healthy': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 25},

'Early Stage Cancer': {'precision': 0.6212121212121212, 'recall': 1.0, 'f1-score': 0.766355140186916, 'support': 41},

'accuracy': 0.6212121212121212

4d. Applying API function to random forest (normalised data)

In [ ]:
# Define the parameter grid? (a dictionary where keys are the hyperparameter names and values are lists of the parameter values to try)
param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]}

rf = RandomForestClassifier(random_state=42, class_weight='balanced')

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, scoring='accuracy', n_jobs=-1)

grid_search.fit(filtered_X_normalized, y_filtered)

# Access the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Use the best model
best_rf = RandomForestClassifier(**grid_search.best_params_)
best_rf.fit(filtered_X_normalized, y_filtered)

# Predict on the test data
y_pred_api = best_rf.predict(filtered_X_normalized_test)

# Calculate accuracy
accuracy = accuracy_score(y_filtered_test, y_pred_api)
print("Random Forest Accuracy for decision tree with API function:", accuracy)

Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Best Score: 0.9084033613445378
Random Forest Accuracy for decision tree with API function: 0.8679706601466992


Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}

Best Score: 0.9084

Random Forest Accuracy for decision tree with API function: 0.8753056234718827

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
y_pred_proba_api = best_rf.predict_proba(filtered_X_normalized_test)[:, 1]
auroc = roc_auc_score(y_filtered_test, y_pred_proba_api)
print("AUROC:", auroc)
#for early stage cancer
precision = precision_score(y_filtered_test, y_pred_api,pos_label='early stage cancer')
recall = recall_score(y_filtered_test, y_pred_api,pos_label='early stage cancer')
f1 = f1_score(y_filtered_test, y_pred_api,pos_label='early stage cancer')
api_evaluation_metrics = classification_report(y_filtered_test, y_pred_api, target_names=['Healthy', 'Early Stage Cancer'], output_dict=True)
print(api_evaluation_metrics)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


AUROC: 0.8105448038176033
{'Healthy': {'precision': 0.9390581717451524, 'recall': 0.9211956521739131, 'f1-score': 0.9300411522633745, 'support': 368}, 'Early Stage Cancer': {'precision': 0.3958333333333333, 'recall': 0.4634146341463415, 'f1-score': 0.42696629213483145, 'support': 41}, 'accuracy': 0.8753056234718827, 'macro avg': {'precision': 0.6674457525392429, 'recall': 0.6923051431601273, 'f1-score': 0.6785037221991029, 'support': 409}, 'weighted avg': {'precision': 0.8846028700950679, 'recall': 0.8753056234718827, 'f1-score': 0.8796106650622246, 'support': 409}}
Precision: 0.9390581717451524
Recall: 0.9211956521739131
F1-score: 0.9300411522633745


AUROC: 0.8105448038176033

Precision: 0.94 for healthy, 0.40 for early stage

Recall: 0.92 for healthy, 0.46 for early stage

F1-score: 0.93 for healthy, 0.43 for early stage

Accuracy: 0.8753056234718827

## 5. Support Vector Machine

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

SVM with linear kernal (baseline)

In [ ]:
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train_pca_filtered, y_filtered)

# Predicting the test set results
y_pred_svm = svm.predict(X_test_pca_filtered)

# Calculating accuracy
accuracy = accuracy_score(y_filtered_test, y_pred_svm)
SVM_evaluation_metrics = classification_report(y_filtered_test, y_pred_svm, target_names=['Healthy', 'Early Stage Cancer'], output_dict=True)
print("Accuracy:", accuracy)
print(SVM_evaluation_metrics)

Accuracy: 0.80

Precision: 0.8 for Healthy, 0.80 for Early Stage

Recall: 0.64 for Healthy, 0.90 for Early Stage

F1-score: 0.71 for Healthy, 0.85 for Early Stage


Implementing Grid Search CV to tune hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf'],
    'degree': [2, 3, 4]}  # Degree parameter for polynomial kernel}

# Instantiate the SVM classifier
svm = SVC(random_state=42)

# Instantiate GridSearchCV
grid_search = GridSearchCV(svm, param_grid, cv=10, scoring='accuracy')

# Perform grid search
grid_search.fit(X_train_pca_filtered, y_filtered)

# Get the best parameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test_pca_filtered)
#accuracy = accuracy_score(y_filtered_test, y_pred, pos_label='early stage cancer')
#precision = precision_score(y_filtered_test, y_pred, pos_label='early stage cancer')
#f1 = f1_score(y_filtered_test, y_pred, pos_label='early stage cancer')
report = classification_report(y_filtered_test, y_pred)
print(report)
print("Best parameters:", best_params)
#print("Accuracy:", accuracy)
#print("Precision:", precision)
#print("F1-score:", f1)

Best parameters: {'C': 100, 'degree': 2, 'kernel': 'linear'}

Accuracy: 0.89

Precision: 0.89 for Healthy, 0.91 for Early Stage

Recall: 0.95 for Healthy, 0.80 for Early Stage

F1-score: 0.92 for Healthy, 0.85 for Early Stage